### PII: Names, locations, organisations - using spacy

#### import packages

In [2]:
import spacy
#from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pandas as pd
import re
from spacy import displacy

#### spacy setup - local

In [3]:
# download to local and then upload to instance
# Using: pytorch  1.13.1 & python 3.10
# n.b. sm means small, lg means large. (n.b. trf is roberta - but doesn't install)

# OR download the .whl and then:
# export PIP_INDEX_URL="https://agile.nat.bt.com/nexus/repository/pypi-proxy/simple"
# pip install en_core_web_lg-3.8.0-py3-none-any.whl
# pip install spacy-transformers=4.37.2

#### packages are installed here: /opt/conda/lib/python3.10/site-packages/

# load spaCy from local:-
# nlp = spacy.load("/opt/conda/lib/python3.10/site-packages/en_core_web_lg/en_core_web_lg-3.8.0")

#### import spacy - local

In [5]:
nlp = spacy.load("/opt/conda/lib/python3.10/site-packages/en_core_web_lg/en_core_web_lg-3.8.0")

#### example dc note

In [4]:
inputText = """
2023-10-18 - Amend
Amend Request received for:<b>CRD</b> - From 22-09-2023 to 19-10-2023, <b>CP Contact Details</b> - From CONTACT NAME: B2B FULFILMENT TEAM B2B FULFILMENT TEAM, CONTACT NO: +448005426754, CONTACT EMAILID: b2bfulfilment.ttt@talktalkplc.com, NOTES: to CONTACT NAME: B2B FULFILMENT TEAM B2B FULFILMENT TEAM, CONTACT NO: +448005426754, CONTACT EMAILID: b2bfulfilment.ttt@talktalkplc.com, NOTES: 

2023-10-19 - General
FND Service Desk C74761750CP ask	Hi Team,End user have accepted the Pinch point, we are unable to amend the Ro Violation.Could you please amend the it , please check the below attachment.

2023-10-19 - HOLDING FACTOR
Holding factor 1651492 allocated to EMP_DELAY_CLOSURES

2023-10-19 - Jeopardy
Complete Site Survey - A end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - General
SI Ref: C74761750 Case Acceptance CCT ID: ONEA10850883 - Communications Provider Name: TALKTALK COMMUNICATIONS LTD  We have reviewed your case and have detailed the next steps below. Case Update Current Status: Good morning, I hope you are well! Thank you for your enquiry.This order is currently with our planning team with delay reference 1651492.Thank you for confirming the customer has accepted the pinch points.I have submitted the delay for closure and I am awaiting a response from the Deemed Consent delay team.Please allow 3 working days and I shall update you accordingly.This case has been allocated to Harjinder Kalair who will update you further by close of play 24/10/23

2023-10-19 - HOLDING FACTOR
Holding factor 1651492 cleared

2023-10-19 - General
B End Design Complete NOTEA 1141    B 1141    CABLE NAME                LENGTH  SUBARC  PAIR   PAIR   PLHGR/0H    HGR/1F     HGR/0H-HGR/1F 1 TIEF        0.01            24          OSHGR/0H    TBISN      HGR/0H-TBISN NO 1           0.04  ON4351    75          OSTBISN     TJBV/A     TBISN-TJBV/A NO 1           0.05  ON4351     3          BSTJBV/A    TJBX/A     TJBV/A-TJBX/A NO 1          0.36  ON4187    75          OSTJBX/A    YZEA/0A    TJBX/A-YZEA/0A NO 1         0.11  ON4187     1          BSB END DESIGN COMPLETE NOTE ONEA10850883CONFIRMED CATEGORY: 1.1TERMINATION FLOOR: GND ROOM: COM SUITE: NA RACK: NADRIVER CIRCUIT: NABASE YCODE: YZEA YCODE: YZEA/A SOF: YZEA/0A TCODE: TJBX/AROUTING: YZEA/0A - TJBX/A - TJBV/A - TBISN - HGR/0HEAD EXISTING: N CIRCUIT ID: NAPSTN EXISTING: YPOWER: ACTEST ROD REQUIRED: N RESOURCE: NASPINE REQUIRED: NNEW LEAD-IN DUCT REQUIRED: NPERMISSION TO WORK/DIG REQUIRED: NPERMISSION TO WORK/DIG OBTAINED: NWAYLEAVE REQUIRED: NARAMS REQUIRED: GENERICPOWER/SPACE CONFIRMED: YNEW INTERNAL CABLE LENGTH REQUIRED: N RESOURCE: NASITE READY FOR INSTALL: YTRAFFIC MANAGEMENT REQUIRED:  None or signingPINCH POINTS (RO2): Y (ACCEPTED)ADDITIONAL COMMENTS: ON SITE CONTACT: MARK SHEPPARD 07976 239835GRANTOR NAME/NUMBER: NAGRANTOR EMAIL: NAGRANTOR ADDRESS: NAInternal Notes/Comments: NAINTERNAL TASKS - ORION UPDATED YINTERNAL TASKS -HOT SITE NINTERNAL TASKS -MOD-S C CLEARANCE REQD NINTERNAL TASKS -COMPATIBLE FOR TCS YINTERNAL TASKS- OVERHEAD / POLE WORK NINTERNAL TASKS - RAILTRACK N

2023-10-19 - Jeopardy
Complete Site Survey - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - ECC
B EndNumber of surveys (No. of units) :: 1 * GBP 298.76 = GBP 298.76Total Excess Construction Charges (GBP) B End = GBP 298.76

2023-10-19 - Jeopardy
Order PSTN/Broadband - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - Jeopardy
Order PSTN/Broadband - A end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - General
The following Duct Lead In Access Violation pinch point has been removed from the list of recorded pinch points. Both circuits share a lead in for the specified distance. Pinch Point Reference - PPA000540270  Exchange - HGR(Haywards Heath) Distance(m) - 64  Location Easting from 533233 to 533167  Location Northing from 124587 to 124578

2023-10-19 - General
The following Internal Cabling Access Violation pinch point has been removed from the list of recorded pinch points. Separate routing within the Customer site premise has not been requested. There is common cable routing within the Customer site.  Pinch Point Reference - PPA000540271  Exchange - HGR(Haywards Heath) Location Easting and Northing - (533233,124587)

2023-10-19 - Jeopardy
Complete Site Survey - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - Jeopardy
Complete Site Survey - A end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - Jeopardy
Complete Site Survey - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - Jeopardy
Complete Site Survey - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - Jeopardy
Complete Site Survey - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - ECC
B EndNumber of surveys (No. of units) :: 1 * GBP 298.76 = GBP 298.76Total Excess Construction Charges (GBP) B End = GBP 298.76

2023-10-19 - ECC
B EndNumber of surveys (No. of units) :: 1 * GBP 298.76 = GBP 298.76Total Excess Construction Charges (GBP) B End = GBP 298.76

2023-10-19 - ECC
B EndNumber of surveys (No. of units) :: 1 * GBP 298.76 = GBP 298.76Total Excess Construction Charges (GBP) B End = GBP 298.76

2023-10-19 - Jeopardy
Order PSTN/Broadband - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - Jeopardy
Assign Access Capacity - A end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - Jeopardy
Order PSTN/Broadband - A end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - Jeopardy
Assign Access Capacity - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - General
ONEA10850883 ORDER PSTN/BB A END TASK **CARD INFILL REQUIRED**A END 1141 exchange code: HGRLOCATION: Floor:1st Floor/ Suite: 104/ Rack:480/ Chassis:20757675/ Slot: 09A END TRS : YMRLS/EA END SOF : HGR/1FA End Network Terminating Equipment : FSP150CM(4U)-DCA END Sub Network Element Identifier : 20757675A END CHASSIS IP: 10.159.96.187A END PSTN: 01444452101A END DVR CCT: ONEA932789Existing Internet Protocol  pool ref number (BTBBMR): BTBBMR271024A END ROUTER SNEID: 1049795

2023-10-19 - General
ONEA10850883 ORDER PSTN/BB B END TASK **NO B END MANAGMENT REQUIREDB END 1141 exchange code: HGRLOCATION: Floor:GND/ Suite:COMMS/ Rack:CAB/ Chassis:/ Slot:1B END TRS : YZEA/AB END SOF : YZEA/0AB End Network Terminating Equipment : FSP150-GE102 ACB END Sub Network Element Identifier : **NEW**NO Public Switched Telephone Network BB REQUIRED

2023-10-19 - Jeopardy
Assign Core Capacity has been set to priority 1 since it has missed its task milestone required by date.

2023-10-20 - Jeopardy
Complete Routing has been set to priority 1 since it has missed its task milestone required by date.

2023-10-20 - General
Complete Routing task - INS/BERT DoneONEA10850883 on INS as Synchronous Digital Hierarchy 3830544INS/BERT COMPLETED AND UPLOADED TO SHAREPOINT FOLDER:-PUBLIC DOCS/ ONEA CCTS/ 2023 10 OCT/ ONEA10000000-ONEA19999999 EST ONLY

2023-10-20 - Jeopardy
Order Stores - A end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-20 - Jeopardy
Order Stores - B end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-20 - Jeopardy
Order VLAN has been set to priority 1 since it has missed its task milestone required by date.

2023-10-20 - HOLDING FACTOR
Holding factor 1657816 raised 
Fundamental Reason: Insufficient Information
Clear Date: 28/02/2024 Time(HH:MM) 23:59
 Target Role: DEEMEDCONSENTDELAY
 Note: Reasonable assistance or information is required from the Communications Provider or end-user, including permits to work/dig Assistance or information required :Order delayed as the sister circuit ONEA10851329 is awaiting for the Communications Provider to accept for the Excess Construction charge charges.Action Owner: CPAdditional information: Order will progress when both the order reaches the same planning stage.When do we expect resolution/next update: Communications Provider DependantThis delay will impact your Customer Committed Date and we intend to apply Deemed Consent. The Customer Committed Date impact will be confirmed when the delay is resolved.
"""

#### redact PERSON tokens

In [5]:
def scrub_pii_with_nlp(text):
    """ use nlp to replace people names with [PERSON]"""
    try:
        nlp_doc = nlp(text)
        final_text = text

        for name in nlp_doc.ents:
            if name.label_ == "PERSON":
                final_text = re.sub(re.escape(name.text), "[PERSON]", final_text)
        outputText = final_text
    except:
        logging.debug(f'cp function error')
    return outputText

In [6]:
# Example usage
redacted_text = scrub_pii_with_nlp(inputText)
print(redacted_text)


2023-10-18 - Amend
Amend Request received for:<b>CRD</b> - From 22-09-2023 to 19-10-2023, <b>CP Contact Details</b> - From CONTACT NAME: B2B FULFILMENT TEAM B2B FULFILMENT TEAM, CONTACT NO: +448005426754, CONTACT EMAILID: b2bfulfilment.ttt@talktalkplc.com, NOTES: to CONTACT NAME: B2B FULFILMENT TEAM B2B FULFILMENT TEAM, CONTACT NO: +448005426754, CONTACT EMAILID: b2bfulfilment.ttt@talktalkplc.com, NOTES: 

2023-10-19 - General
FND Service Desk C74761750CP ask	Hi Team,End user have accepted the Pinch point, we are unable to amend the Ro Violation.Could you please amend the it , please check the below attachment.

2023-10-19 - HOLDING FACTOR
Holding factor 1651492 allocated to EMP_DELAY_CLOSURES

2023-10-19 - Jeopardy
Complete Site Survey - A end has been set to priority 1 since it has missed its task milestone required by date.

2023-10-19 - General
SI Ref: C74761750 Case Acceptance CCT ID: ONEA10850883 - Communications Provider Name: TALKTALK CO[PERSON]UNICATIONS LTD  We have reviewed

#### explore entities

In [7]:
print(f'PERSON - {spacy.explain("PERSON")}')
print(f'GPE    - {spacy.explain("GPE")}')
print(f'DATE   - {spacy.explain("DATE")}')
print(f'MONEY  - {spacy.explain("MONEY")}')
print(f'ORG  - {spacy.explain("ORG")}')
print(f'CARDINAL  - {spacy.explain("CARDINAL")}')
print(f'PRODUCT  - {spacy.explain("PRODUCT")}')

PERSON - People, including fictional
GPE    - Countries, cities, states
DATE   - Absolute or relative dates or periods
MONEY  - Monetary values, including unit
ORG  - Companies, agencies, institutions, etc.
CARDINAL  - Numerals that do not fall under another type
PRODUCT  - Objects, vehicles, foods, etc. (not services)


In [8]:
doc = nlp(inputText)

In [9]:
[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]

[('2023-10-18 - Amend', 1, 19, 'DATE'),
 ('B2B FULFILMENT TEAM B2B FULFILMENT TEAM', 138, 177, 'ORG'),
 ('B2B FULFILMENT TEAM B2B FULFILMENT TEAM', 282, 321, 'ORG'),
 ('2023-10-19 - General', 411, 431, 'DATE'),
 ('FND Service Desk C74761750CP', 432, 460, 'ORG'),
 ('the Ro Violation', 536, 552, 'LAW'),
 ('2023-10-19 - HOLDING FACTOR\nHolding', 621, 656, 'DATE'),
 ('1651492', 664, 671, 'CARDINAL'),
 ('2023-10-19', 705, 715, 'DATE'),
 ('1', 781, 782, 'CARDINAL'),
 ('2023-10-19 - General', 841, 861, 'DATE'),
 ('TALKTALK COMMUNICATIONS LTD  ', 949, 978, 'ORG'),
 ('3 working days', 1395, 1409, 'DATE'),
 ('Harjinder Kalair', 1477, 1493, 'ORG'),
 ('2023-10-19 - HOLDING FACTOR\nHolding', 1549, 1584, 'DATE'),
 ('1651492', 1592, 1599, 'DATE'),
 ('2023-10-19 - General', 1609, 1629, 'DATE'),
 ('1141', 1658, 1662, 'CARDINAL'),
 ('1141', 1668, 1672, 'CARDINAL'),
 ('HGR/0H', 1755, 1761, 'ORG'),
 ('0.01', 1783, 1787, 'CARDINAL'),
 ('24', 1799, 1801, 'CARDINAL'),
 ('HGR/0H', 1834, 1840, 'ORG'),
 ('1    

#### visualise entities

In [10]:
# visualise
displacy.render(doc,style='ent',jupyter=True)

#### understanding tokens

In [11]:
tokens = nlp("dog cat banana <EMAIL>")

for token in tokens:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

dog True 7.0336733 False
cat True 6.6808186 False
banana True 6.700014 False
< True 6.446124 False
EMAIL True 6.915451 False
> True 6.889212 False


#### similarity - ensure use a larger spacy package e.g. lg

In [12]:
doc1 = nlp("I like salty fries and hamburgers.")
doc2 = nlp("Fast food tastes very good.")

# Similarity of two documents
print(doc1, "<->", doc2, doc1.similarity(doc2))
# Similarity of tokens and spans
french_fries = doc1[2:4]
burgers = doc1[5]
print(french_fries, "<->", burgers, french_fries.similarity(burgers))

I like salty fries and hamburgers. <-> Fast food tastes very good. 0.7687607012190486
salty fries <-> hamburgers 0.6949787735939026
